In [1]:
from train import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current device:", device)

Current device: cuda


In [3]:
def lhs(size, lb=-1, rb=1):
    res = np.zeros(size)
    length = (rb - lb) / size
    for i in range(size):
        sub_lb = lb + length * i
        sub_rb = lb + length * (i + 1)
        res[i] = np.random.rand() * length + sub_lb
    return res
        

In [ ]:
epochs = 10000
lr = 0.001

alpha = np.random.uniform(low=0.005/np.pi, high=0.1/np.pi, size=1000)
# alpha = np.random.uniform(low=-1, high=1, size=10000)
# beta = np.random.uniform(low=-1, high=1, size=1000)
# print("Alpha: {:.3f}, Beta: {:.3f}".format(alpha, beta))
# print(alpha)
# print(beta)
# loss_i, loss_b, loss_f, loss, model, val_loss, val_ood_loss, nrmse = train(epochs=epochs, lr=lr, i_size=0, b_size=2, f_size=1000, zero_shot=False, alpha_list=0.6, beta_list=-0.3, low=-10, high=10, eqname='poisson')
# loss_i, loss_b, loss_f, loss, model, val_loss, val_ood_loss, nrmse = train(epochs=epochs, lr=lr, i_size=0, b_size=2, f_size=1, zero_shot=True, alpha_list=alpha, beta_list=beta, low=-1, high=1, eqname='poisson')
loss_i, loss_b, loss_f, loss, model, val_loss, val_ood_loss, nrmse = train(epochs=epochs, lr=lr, i_size=1, b_size=2, f_size=1, zero_shot=True, alpha_list=alpha, low=-1, high=1, eqname='burgers')
# loss_i, loss_b, loss_f, loss, model, val_loss, val_ood_loss, nrmse = train(epochs=epochs, lr=lr, i_size=100, b_size=100, f_size=10000, zero_shot=False, alpha_list=(0.01 / np.pi), low=-1, high=1, eqname='burgers')

Current device: cuda
Data generation completed


In [ ]:
epochs = [50 * i for i in range(1, 21)]
plt.plot(epochs, val_loss, label='Val.')
plt.plot(epochs, val_ood_loss, label='Val. OOD')
# plt.yscale('log')
plt.title("Zero-shot Poisson")
plt.legend()
plt.show()

In [ ]:
test_x = np.linspace(-10, 10, num=100).reshape(-1, 1)
# test_alpha = np.full((100, 1), alpha[2])
# test_beta = np.full((100, 1), beta[2])
test_alpha = np.full((100, 1), 0.6)
test_beta = np.full((100, 1), -0.3)
# test_in = np.hstack((test_x, test_alpha, test_beta))
test_in = test_x
test_u = model(torch.Tensor(test_in).to(device))
X = test_x

Y = np.sin(test_alpha * X) + np.cos(test_beta * X) + 0.1 * X


In [ ]:
C = test_u.cpu().detach().numpy()
plt.figure(figsize=(10, 8))
plt.scatter(X, C, label='PINN')
plt.scatter(X, Y, label='Answer')
plt.legend()
plt.colorbar()

In [ ]:
np.sqrt(np.sum( (Y-C) ** 2) / np.sum( Y ** 2 ))

In [ ]:
from burgers import *

In [ ]:
if model == None:
    model = PINN(20, 8, dim=2, zero_shot=False)
    model.load_state_dict(torch.load('burgers.data'))
    model.to(device)


In [ ]:
vtn = 101
vxn = 101
nu = 0.318310E-02
vx = np.linspace(-1, 1, vxn)
vt = np.linspace(0, 1, vtn)
x, t = np.meshgrid(vx, vt)
x = x.reshape(-1, 1)
t = t.reshape(-1, 1)
alpha = np.full((x.shape), nu)
pred = model(torch.Tensor(np.hstack((x, t, alpha))).to(device)).detach().cpu().numpy()
# pred = model(torch.Tensor(np.hstack((x, t))).to(device)).detach().cpu().numpy()
truth = burgers_viscous_time_exact1(nu, vxn, vx, vtn, vt).T

In [ ]:
x_025 = np.linspace(-1, 1, 101).reshape(-1, 1)
t_025 = np.full((x_025.shape), 0.5)
pred = model(torch.Tensor(np.hstack((x_025, t_025))).to(device)).detach().cpu().numpy()

In [ ]:
plt.scatter(x, t, c=pred, cmap='seismic')
plt.colorbar()

In [ ]:
plt.scatter(x, t, c=truth, cmap='seismic')
plt.colorbar()

In [ ]:
print(vt[50])
plt.plot(truth[50, :], label='truth')
plt.plot(pred, label='pred')
plt.legend()

In [ ]:
np.sqrt(np.sum((pred-truth.reshape(-1, 1)) ** 2) / np.sum(pred ** 2))

In [ ]:
np.mean(np.abs(pred-truth.reshape(-1, 1)))

In [ ]:
plt.scatter(x, t, c=((pred.reshape(101, 101) - truth) ** 2).reshape(-1, 1), cmap='seismic')
plt.colorbar()